<a href="https://colab.research.google.com/github/movie112/INU-DILAB/blob/main/210825_210831/word2vec_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Distributed Representations of Words and Phrases and their Compositionality
- 2013


## abstract
- continuous skip-gram이 정확한 syntatic, sementic 단어 관계를 포착하는 분산 표현 학습에 효율적
- 이 논문에서, 벡터 품질 향상, training speed up에 대한 몇 가지 확장을 보여준다.
  - 빈도수에 대한 subsampling -> speed up, more reqular word representation
  - negative sampling은 hierarchical(계층적) softmax를 대체

>- 단어 표현의 본질적(내제적) 한계
>  - 어순에 대한 무관심
>  - 관용구 표현의 무능
>    - "Air Canada" 같은 문장에 있어도 연관성이 없어서 학습할 때 -요소
>    - 둘을 묶어서 하나의 word로 봤더니 결과가 좋더라



## 1. Introduce
- in vector space, 분산표현은 유사 단어를 묶어서 NLP performance up시키는 알고리즘 학습에 도움이 된다.
- 1986-통계 언어 모델링-자동화자인식, 기계번역-wide range NLP
> - skip-gram model: 많은 양의 text data의 high quality vector representation learning에 효울적인 방법이다.
>   - dense matrix 곱셈을 안해서 효율적
>   - 1개 머신이 하루에 100billion 단어를 학습할 수 있다.

- 신경망으로 계산되는 단어표현에서 학습된 벡터는 언어 규칙과 패턴을 encode한다.
- 많은 패턴들은 선형 변환으로 표현될 수 있다.
  - vec("Madrid")-vec("Spain")+vec("France")는 vec("Paris")에 가까워 진다.

> - original skip-gram의 확장 몇 가지
>   - train 동안 빈도수에 대한 __subsampling__은 speedup, 적은 빈도인 단어표현의 accuracy improve
>   - skip-gram model을 train할 때 __Noise Contrastive Estimation(NCE)__의 단순한 변형을 제시한다.
>     - 이전에 사용했던 hierarchhical softmax보다 더 학습 빠르고, 더 나은 단어표현

- 단어 표현의 한계
  - 관용 구절은 개별 단어들의 합성이 아니어서 표현할 수 없다
  - ex) 기사에서 "Boston" and "Globe"의 조합은 의미상 자연스럽지 않다. 이를 하나의 구절로 나타내면 skip-gram 모델의 표현이 더 쉬워진다.
    - 문장의 의미를 표현하는 목표를 가진 다른 기법들도 word vector 보다 phrase vector을 사용하는 것이 이점이 있을 것이다.

> - word based -> phrase based model extension
>   - 많은 수의 구문을 식별하고
>   - 학습하는 동안 구문을 개별 토큰으로 처리
>   - 구문 벡터를 평가할 때, 단어와 구문을 포함하는 아날로그 추론 task의 task set을 develop 했다.
>     - 아날로그 쌍: "Montreal" : "Montreal Canadiens" :: "Toronto" : "Toronto Maple Leafs"
>     - vec("Montreal Canadiens") - vec("Montreal") + vec("Toronto") = vec("Toronto Maple Leafs")

- 또 다른 특성
  - vec("Germany") + vec("capital") = vec("Berlin")
- 기초적인 수학 연산으로 명확하지 않은 정도의 언어 이해를 얻을 수 있다.

---
## 2. The Skip-gram Model
- skip-gram의 학습 목적은 문장이나 문서에서 주변단어를 예측하는 유용한 단어 표현을 찾는 것
- sequence of training words(w1, w2, ..., wn)가 주어졌을 때, skip-gram의 목적은 average log 확률을 최대화하는 것

> - __function(1)__
>   - c: size of the training context(center word wt기준)
>   - 큰 window size는 학습 사례를 더 많이하고, 더 높은 정확도를 이끈다.
> - 기본 skip-gram 공식은 softmax function을 이용하여 P(wt+j|wt) 정의한다.
> - __function(2)__
>   - vw / v'w는 input / output vector reprewentations of w
>   - W는 number of words in the vocabulary
>   - 이 formulationdms impractical 함
>     - cost 계산이 ▽logP(wo|wi) W에 비례해서(하나 예측하려면 W비율로 다 계산해야 함)


### 2.1 Hierarchical softmax
- full softmax에서 계산 상 효율적인 근사치(approximation)은 hierarchical softmax 이다.
- advantage
  - 확률분포를 얻기 위해 신경망의 output nodes의 W를 평가하는 대신 __log2(W)__를 평가 함

> - binary tree를 이용 of the output layer
>   - W의 단어들이 leafs, 각 노드들은 하위 노드들과의 상대적인 확률을 명시
>   - 이것들은 단어에 확률을 할당하는 random walk를 정의

- 각 단어 w는 root로 부터 도달할 수 있다.
- n(w,j)는 root부터 w까지의 j번 째노드, L(w)는 path의 길이
- n(w,1)=root, n(w,L(w))=w
- ch(n)은 n의 임의의 고정된 하위 노드
- [x]는 if x is true =>1 and -1 otherwise
- hierarchical softmax는 __function(3)__로 P(wo|wi)를 정의
- __function(3)__

> -  σ(x) = 1/(1+exp(-x))
>   - 이는 ΣP(w|wi)=1 를 확인할 수 있다.
>   - 이는 log p(wo|wi) 계산 cost 와 L(wo)에 비례하는 ▽logP(wo|wi)을 내포한다.(+ 평균적으로 logW보다 크지 않음)
> 각 단어w 마다 두 개의 표현(vw, v'w)을 할당하는 skip-gram의 표준 softmax 공식과 달리 __hierarchical softmax__는 각 단어w에 대해 vw, 이진트리의 모든 내부 노드n에 대해 v'n을 가진다.

- hierarchical softmax에 사용되는 트리 구조는 performence 상당한 효과를 보인다.
- in out work we use a __binary Huffman tree__: 잦은 단어에 짧은 코드를 할당해서 train이 빠름
  - 이전에 이것이 빈도수에 대해 단어를 묶는 것이 신경망 언어모델에서 간단하게 speedup되는 기술임을 알았다.
  


### 2.2 Nagative Sampling
- hierarchical softmax의 대안은 __Noise Contrastive Estimation(NCE)__이다.
  - NCE는 좋은 모델이란 logistic repression의 평균에 대해 noise를 데이터와 구별해야 한다고 가정한다.
  - 이것은 noise에 순위를 매겨 학습된 모델인 hinge loss와 비슷하다.

> - NCE는 sotfmas의 log 확률을 최대화하는 것을 보일 수 있다.
>   - skip-gram모델은 오직 learning high-quality vector representations을 고려함
>   - qulity를 유지한다면 자유롭게 NCE를 단순화할 수 있다.
> - 우리는 __negative sampling__을 objective로 정의한다.
> - __function(4)__

- skip-gram objective에서 모든 logP(wo|wi) term을 대체하는 데 사용된다.
- 따라서 task는 target word wo와 noise 분포Pn(w)를 구분하는 것이다. 회귀 분석 사용해서
- data sample 마다 k negative samples 있다.
  - k가 5-20 사이이면 small training datasets에 유용하고
  - 2-5면, large datasets에서 유용하다.
- negative sampling과 NCE가 다른 점
  - NCE는 두 개의 sample과 noise 분포의 numerical 확률이 필요하다.
  - negative sampling은 오직 sample만 사용
  - 그리고 NCE가 softmax의 log 확률을 최대화하지만 이것은 우리에게 중요하지 않다.

> - NCE, NEG 모두 free parameter로 noise distiribution Pn(w)를 가진다.
> - 우리는 Pn(w)의 선택지의 수를 조사하였고, 3/4승 향상된 unigram 분포 U(w)를 찾았는데 NCE, NEG에서 unigram distribution은 언어모델을 포함한 모든 task에서 unigram과 균일분포(어떤 곳에서든 같은 값을 가짐)를 상당히 능가함




### 2.3 Subsampling of Frequent Words
- 큰 corpora에서 빈도수 엄청 많이 나오는 단어들(관사같은거)은 rare word보다 적은 정보를 제공한다.
- 예시, 반면 skip-gram은 "Paris"와 "France" co-occurrences를 관찰해서 이익을 얻고, "France"와 "the"의 co-occurrence 관찰은 덜 유익하다.
- "the"같은 건 어느 문장에서나 자주 등장한
  - 이 아이디어를 반대로도 적용할 수 있다.
  - 빈번한 단어의 벡터표현은 수백만 개의 예제를 학습한 후에도 많이 변하지 않는다.

> - 희귀 단어와 빈번한 단어 간 불균형에 대응하기 위해 우리는 단순한 __subsampling__ 접근 방식을 이용한다.
  > - training set 내 각 단어 wi는 공식에 의해 계산된 확률로 버려진다.
> - __function(5)__

- f(wi)는 word wi의 빈도수, t는 선택된 임계값- 일반적으로 10^-5
- 이 subsampling 공식을 고른 이유는 빈도수의 순위를 유지하면서 t보다 큰 단어를 적극적으로 subsample해서 이다.
- 이 subsampling formula가 heuristically(스스로) 선택되었으나, 우리는 이것이 잘 작동한다는 것을 알았다.
- 이것은 학습을 가속화하고 rare words 학습 벡터의 정확도가 상당히 향상시킨다.(다음 섹센에서 보여줌)